In [1]:
import pandas as pd
import importlib
import fileDistanceProcessing as fdp
import importlib
importlib.reload(fdp) 

<module 'fileDistanceProcessing' from '/home/marko/Desktop/ip2/ip2_projekat/source/fileDistanceProcessing.py'>

In [2]:
X=pd.read_csv("../data/csv/sars2_mers_sars1.txt",skipinitialspace = True )

In [3]:
X.columns = X.columns.str.strip()
X.columns

Index(['Virus', 'ACCESSION', 'WHO_TYPE', 'ACCESSION_PROT', 'PROTEIN',
       'DUZINA_NUC', 'KOD_NUC'],
      dtype='object')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [5]:
distances = fdp.deserialize_and_decompress_distance_matrix("../data/full_distances_matrix.7z")

In [7]:
# metrika zasnovana samo na 5-adic distanci
# instant racunanje iz matrice
def five_adic_distance_from_matrix(x, y):
    
    i = int(x[-1]) #indeks je na poslednjem mestu u listi
    j = int(y[-1])

    # matrica je u obliku trougla bez dijagonale originalne matrice
    first = min(i, j)
    second = max(i, j) - (first + 1) # veci indeks smanji za broj instanci u redu matrice do dijagonale 

    return 0 if i == j else distances[first][second] #udaljenost x od y

In [8]:
y_virus=X["Virus"]
X_virus=[] # potreban nam je samo indeks

# na svaku instancu X_virus smo appendovali njen index
for i in range(len(y_virus)):
    X_virus.append([0, i]) # 0 je dodata zbog knn da ne pravi problem

In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
param_grid = {'n_neighbors': range(2, 10),
              'metric': [five_adic_distance_from_matrix]}

In [11]:
from sklearn.neighbors import KNeighborsClassifier

In [12]:
gs = GridSearchCV(KNeighborsClassifier(),
             param_grid=param_grid,
             scoring='accuracy',
             cv=4,
             verbose=5)

In [14]:
gs.fit(X_virus, y_virus)

Fitting 4 folds for each of 8 candidates, totalling 32 fits
[CV 1/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.996 total time=  40.2s
[CV 2/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.998 total time=  42.2s
[CV 3/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.998 total time=  45.1s
[CV 4/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.998 total time=  47.8s
[CV 1/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=3;, score=0.996 total time=  40.3s
[CV 2/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=3;, score=0.998 total time=  42.4s
[CV 3/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=3;, score=0.998 total time=  44.8s
[CV 4/4] END metric=<function five_adic_dist

GridSearchCV(cv=4, estimator=KNeighborsClassifier(),
             param_grid={'metric': [<function five_adic_distance_from_matrix at 0x7f7b302abf70>],
                         'n_neighbors': range(2, 10)},
             scoring='accuracy', verbose=5)

In [17]:
y_protein = X["PROTEIN"]
X_protein = X_virus

In [18]:
gs.fit(X_protein, y_protein)

Fitting 4 folds for each of 8 candidates, totalling 32 fits


/home/marko/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=4.
  warnings.warn(


[CV 1/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.995 total time=  37.2s
[CV 2/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.996 total time=  39.3s
[CV 3/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.996 total time=  42.5s
[CV 4/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.994 total time=  45.9s
[CV 1/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=3;, score=0.999 total time=  37.2s
[CV 2/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=3;, score=0.999 total time=  39.8s
[CV 3/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=3;, score=0.999 total time=  42.5s
[CV 4/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=3;, score=0

GridSearchCV(cv=4, estimator=KNeighborsClassifier(),
             param_grid={'metric': [<function five_adic_distance_from_matrix at 0x7f7b302abf70>],
                         'n_neighbors': range(2, 10)},
             scoring='accuracy', verbose=5)

In [19]:
import numpy as np

virusTypes = np.array(X['Virus'])
X_who = []
indexes = []
for i in range(len(virusTypes)):
    if(virusTypes[i].strip() == "SARS_COV_2"):
        X_who.append([0, i])
        indexes.append(i)

sars2_only = X.filter(items=indexes, axis=0)

In [20]:
y_who=sars2_only["WHO_TYPE"].str.strip()

In [21]:
gs.fit(X_who, y_who)

Fitting 4 folds for each of 8 candidates, totalling 32 fits
[CV 1/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.893 total time=  31.6s
[CV 2/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.966 total time=  32.1s
[CV 3/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.968 total time=  32.4s
[CV 4/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=2;, score=0.966 total time=  32.2s
[CV 1/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=3;, score=0.894 total time=  31.5s
[CV 2/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=3;, score=0.967 total time=  32.2s
[CV 3/4] END metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>, n_neighbors=3;, score=0.974 total time=  32.3s
[CV 4/4] END metric=<function five_adic_dist

GridSearchCV(cv=4, estimator=KNeighborsClassifier(),
             param_grid={'metric': [<function five_adic_distance_from_matrix at 0x7f7b302abf70>],
                         'n_neighbors': range(2, 10)},
             scoring='accuracy', verbose=5)

In [22]:
gs.best_estimator_

KNeighborsClassifier(metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>,
                     n_neighbors=4)

In [23]:
gs.best_estimator_.fit(X_who, y_who)

KNeighborsClassifier(metric=<function five_adic_distance_from_matrix at 0x7f7b302abf70>,
                     n_neighbors=4)

In [25]:
y_pred = gs.best_estimator_.predict(X_who)

In [26]:
accuracy_score(y_who, y_pred)

0.9802347127856702

In [69]:
print(confusion_matrix(y_who,y_pred, labels=y_who.unique()))


[[1474    1    2    2    5    0   13    0    3    0    0    0    0]
 [   8 1475    2    0    5    4    0    1    4    0    1    0    0]
 [   1    8 1480    1    1    0    0    0    8    0    0    1    0]
 [   3    1    4 1479    3    3    0    0    4    1    0    1    1]
 [   2    1    2    2 1487    2    0    0    2    1    1    0    0]
 [   5    9    6    2    3 1464    5    0    4    0    2    0    0]
 [   1    3    0    0    0    0  458    1    4    0    0    0    0]
 [   1    6    0    0    0    1    0  148    1    0    0    0    0]
 [   1    1    5    3    0    0    0    1  575    0    0    0    0]
 [   4    3    0    0    2    2    2    0    4  611    0    0    0]
 [   2    1    3    0    3    0    4    0    8    1  388    0    0]
 [   0    1    7    0    1    0    0    0    1    0    0   55    0]
 [   0    1    0    0    0    1    1    0    1    1    0    0   15]]


In [54]:
conf_mat = confusion_matrix(y_who,y_pred, labels=y_who.unique())
labels = y_who.unique()
class_error_rate = []
for i in range(len(conf_mat)):
    correct = 0
    incorrect = 0
    for j in range(len(conf_mat[i])):
        if i == j:
            correct += conf_mat[i][j]
        else:
            incorrect += conf_mat[i][j]
    class_error_rate.append({labels[i], format(correct / (correct + incorrect), '.4f')})
class_error_rate

[{'0.9827', 'Alpha'},
 {'0.9833', 'Delta'},
 {'0.9867', 'Epsilon'},
 {'0.9860', 'Gamma'},
 {'0.9913', 'Iota'},
 {'0.9760', 'Omicron'},
 {'0.9807', 'Zeta'},
 {'0.9427', 'Kappa'},
 {'0.9812', 'Beta'},
 {'0.9729', 'Eta'},
 {'0.9463', 'Lambda'},
 {'0.8462', 'Mu'},
 {'0.7500', 'Theta'}]